### Charging Profile

In [ ]:
from scipy.optimize import fsolve
import numpy as np

ev_mpi = 7.36
soc = 0.47
d_c = 0.9
cap = 70
k = 0.3  # 0.3 for better battery health

# --------------------------------------------------------------------------
current_charge = soc * cap

charge_1 = 0.8 * cap - current_charge  # first part of charge < 80% of cap
charge_2 = d_c * cap - 0.8 * cap  #      second part of charge > 80% of cap

cvt = 0.8 * cap / ev_mpi  # time to charge till 80% with CC, change to CV
print(charge_1, charge_2)


# Define the function for the given equation
def equation(t):
    return ev_mpi * (t - cvt) * np.exp(-k * (t - cvt)) - charge_2


# Initial guess
initial_guess = cvt

# Solve the equation numerically
solution = fsolve(equation, initial_guess)
print([cvt, ev_mpi], [solution[0] - cvt, charge_2 / (solution[0] - cvt)]  )

print(equation(solution[0]) + charge_2 + charge_1 + soc * cap, d_c * cap)
